# Basic face recognizer using pre trained model

Implemented a Face recognizer to identify person showing on a web cam.
,Model = FaceNet Inception model,
Weights = pretrained Keras-OpenFace minimised triplet loss weights,
Auto crops faces from given input images for training.

## Import libraries



In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import sys
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

## FaceNet's Inception model

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

## Keras-OpenFace pretrained weights by minimizing triplet loss

In [3]:
weights = utils.weights
weights_dict = utils.load_weights()

for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## Create embedding vector for images (a)

In [4]:
def image_to_embedding(image, model):
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


In [5]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.60:
        return str(name)
    else:
        return None

## Create embedding vector for images (b)

In [6]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Function to auto-crop faces from training images

In [11]:
import os
import glob
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
count = 0
for file in glob.glob("train/*/*/*.jpg"):
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        count += 1   
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.imwrite("/home/suj/Desktop/FR_1_shot_learning/images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])

## Live Face Recognition 

In [ ]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

distance from User_4 is 1.0071615
distance from User_5 is 1.1736356
distance from User_48 is 0.93312067
distance from User_27 is 1.1168283
distance from User_53 is 1.3703473
distance from User_43 is 0.9493447
distance from User_38 is 1.196937
distance from User_24 is 1.3753998
distance from User_32 is 1.3808763
distance from User_45 is 1.024711
distance from User_42 is 1.1526098
distance from User_60 is 0.9609663
distance from User_39 is 0.93312067
distance from User_40 is 1.1041156
distance from User_49 is 1.0559145
distance from User_26 is 1.0248069
distance from User_14 is 1.0322025
distance from User_31 is 0.99935424
distance from User_57 is 1.2025172
distance from User_56 is 1.3177173
distance from User_11 is 0.9088748
distance from User_51 is 1.1083286
distance from User_23 is 1.068064
distance from User_3 is 0.96048784
distance from User_55 is 1.2239037
distance from User_59 is 0.9296881
distance from User_54 is 1.1970112
distance from User_58 is 1.3484763
distance from User_7 i

distance from User_4 is 1.0859799
distance from User_5 is 1.2308933
distance from User_48 is 0.88788337
distance from User_27 is 1.1651864
distance from User_53 is 1.3551306
distance from User_43 is 0.8951059
distance from User_38 is 1.310493
distance from User_24 is 1.4169154
distance from User_32 is 1.4084942
distance from User_45 is 1.0525097
distance from User_42 is 1.1724863
distance from User_60 is 1.0532227
distance from User_39 is 0.88788337
distance from User_40 is 1.1072009
distance from User_49 is 1.186458
distance from User_26 is 1.1177975
distance from User_14 is 1.0892615
distance from User_31 is 1.02255
distance from User_57 is 1.271457
distance from User_56 is 1.3003637
distance from User_11 is 0.8168556
distance from User_51 is 1.1336946
distance from User_23 is 1.2289497
distance from User_3 is 0.8246284
distance from User_55 is 1.2366315
distance from User_59 is 0.9436181
distance from User_54 is 1.2070396
distance from User_58 is 1.3747482
distance from User_7 is 0.

distance from User_4 is 1.1555767
distance from User_5 is 1.2749746
distance from User_48 is 0.95040065
distance from User_27 is 1.1945357
distance from User_53 is 1.4173657
distance from User_43 is 0.9778454
distance from User_38 is 1.3208295
distance from User_24 is 1.4794561
distance from User_32 is 1.4454422
distance from User_45 is 1.0907627
distance from User_42 is 1.2673016
distance from User_60 is 1.096067
distance from User_39 is 0.95040065
distance from User_40 is 1.2030456
distance from User_49 is 1.1847004
distance from User_26 is 1.1752707
distance from User_14 is 1.1386102
distance from User_31 is 1.061754
distance from User_57 is 1.3160806
distance from User_56 is 1.3714058
distance from User_11 is 0.8495883
distance from User_51 is 1.1853861
distance from User_23 is 1.2088385
distance from User_3 is 0.86414903
distance from User_55 is 1.301303
distance from User_59 is 1.0149583
distance from User_54 is 1.2716491
distance from User_58 is 1.4033985
distance from User_7 is

distance from User_4 is 1.1982601
distance from User_5 is 1.3278476
distance from User_48 is 0.968877
distance from User_27 is 1.1906928
distance from User_53 is 1.4348098
distance from User_43 is 1.0120411
distance from User_38 is 1.3266716
distance from User_24 is 1.4646941
distance from User_32 is 1.4514043
distance from User_45 is 1.0823879
distance from User_42 is 1.2923352
distance from User_60 is 1.1056587
distance from User_39 is 0.968877
distance from User_40 is 1.1921127
distance from User_49 is 1.1879724
distance from User_26 is 1.1935103
distance from User_14 is 1.1709487
distance from User_31 is 1.1197455
distance from User_57 is 1.3525656
distance from User_56 is 1.386349
distance from User_11 is 0.8462665
distance from User_51 is 1.2444019
distance from User_23 is 1.2125053
distance from User_3 is 0.8434034
distance from User_55 is 1.338402
distance from User_59 is 1.0128299
distance from User_54 is 1.3148851
distance from User_58 is 1.4182425
distance from User_7 is 1.0

distance from User_4 is 1.1180384
distance from User_5 is 1.1979814
distance from User_48 is 0.9239277
distance from User_27 is 1.2105252
distance from User_53 is 1.3930545
distance from User_43 is 0.9470819
distance from User_38 is 1.2846544
distance from User_24 is 1.4539071
distance from User_32 is 1.3874567
distance from User_45 is 1.0640036
distance from User_42 is 1.2244397
distance from User_60 is 1.0505693
distance from User_39 is 0.9239277
distance from User_40 is 1.1476539
distance from User_49 is 1.1258667
distance from User_26 is 1.159059
distance from User_14 is 1.0297415
distance from User_31 is 1.047163
distance from User_57 is 1.2410214
distance from User_56 is 1.3389001
distance from User_11 is 0.8719366
distance from User_51 is 1.1230748
distance from User_23 is 1.176374
distance from User_3 is 0.85859317
distance from User_55 is 1.2249942
distance from User_59 is 0.985235
distance from User_54 is 1.2463199
distance from User_58 is 1.3448088
distance from User_7 is 0.

...

distance from User_4 is 0.7255414
distance from User_5 is 1.017245
distance from User_48 is 0.94842297
distance from User_27 is 1.0900193
distance from User_53 is 1.1251038
distance from User_43 is 0.96295786
distance from User_38 is 0.8076725
distance from User_24 is 0.89255154
distance from User_32 is 0.8514707
distance from User_45 is 0.81904316
distance from User_42 is 0.79593825
distance from User_60 is 0.6323616
distance from User_39 is 0.94842297
distance from User_40 is 0.7479366
distance from User_49 is 0.8217244
distance from User_26 is 0.75298226
distance from User_14 is 0.7401606
distance from User_31 is 0.83300626
distance from User_57 is 0.838559
distance from User_56 is 1.0884998
distance from User_11 is 1.0232124
distance from User_51 is 1.0322585
distance from User_23 is 0.8592924
distance from User_3 is 1.0631919
distance from User_55 is 0.962624
distance from User_59 is 0.7775199
distance from User_54 is 0.9728407
distance from User_58 is 1.0517337
distance from User

distance from User_4 is 0.8745919
distance from User_5 is 1.0546546
distance from User_48 is 1.0107814
distance from User_27 is 0.8991026
distance from User_53 is 1.1298748
distance from User_43 is 1.0226635
distance from User_38 is 1.0081213
distance from User_24 is 0.9648389
distance from User_32 is 0.9604969
distance from User_45 is 0.8772001
distance from User_42 is 0.90190893
distance from User_60 is 0.7493665
distance from User_39 is 1.0107814
distance from User_40 is 0.755157
distance from User_49 is 0.86741954
distance from User_26 is 0.83150876
distance from User_14 is 0.8226981
distance from User_31 is 0.8958117
distance from User_57 is 0.9353704
distance from User_56 is 1.0923975
distance from User_11 is 1.0681202
distance from User_51 is 1.1131443
distance from User_23 is 0.7243052
distance from User_3 is 1.0991597
distance from User_55 is 1.0306675
distance from User_59 is 0.7599906
distance from User_54 is 1.0775126
distance from User_58 is 1.0616571
distance from User_7 

distance from User_4 is 0.71408397
distance from User_5 is 0.9708221
distance from User_48 is 0.8594712
distance from User_27 is 1.0080272
distance from User_53 is 1.0494081
distance from User_43 is 0.82306147
distance from User_38 is 0.85897034
distance from User_24 is 0.84405005
distance from User_32 is 0.93323827
distance from User_45 is 0.67651486
distance from User_42 is 0.8031079
distance from User_60 is 0.5932095
distance from User_39 is 0.8594712
distance from User_40 is 0.59120625
distance from User_49 is 0.7576059
distance from User_26 is 0.7271187
distance from User_14 is 0.74072075
distance from User_31 is 0.7817749
distance from User_57 is 0.84222245
distance from User_56 is 1.0026286
distance from User_11 is 0.9156635
distance from User_51 is 0.980346
distance from User_23 is 0.73370934
distance from User_3 is 0.9258695
distance from User_55 is 0.9811609
distance from User_59 is 0.6003189
distance from User_54 is 0.9861125
distance from User_58 is 0.9907632
distance from 

distance from User_4 is 0.8307459
distance from User_5 is 1.0933218
distance from User_48 is 1.0763605
distance from User_27 is 0.9944499
distance from User_53 is 1.1630791
distance from User_43 is 1.1062394
distance from User_38 is 0.761786
distance from User_24 is 0.7235589
distance from User_32 is 0.87620425
distance from User_45 is 0.8702598
distance from User_42 is 0.7646743
distance from User_60 is 0.7419948
distance from User_39 is 1.0763605
distance from User_40 is 0.8078144
distance from User_49 is 0.79168284
distance from User_26 is 0.7511194
distance from User_14 is 0.8339115
distance from User_31 is 0.9736678
distance from User_57 is 0.90601975
distance from User_56 is 1.1366588
distance from User_11 is 1.1814585
distance from User_51 is 1.095434
distance from User_23 is 0.80414414
distance from User_3 is 1.2284667
distance from User_55 is 1.0439912
distance from User_59 is 0.8563763
distance from User_54 is 1.0023092
distance from User_58 is 1.0764142
distance from User_7 

distance from User_4 is 0.9882259
distance from User_5 is 1.2003002
distance from User_48 is 1.1123046
distance from User_27 is 1.0118166
distance from User_53 is 1.3056285
distance from User_43 is 1.1412729
distance from User_38 is 0.8496181
distance from User_24 is 0.8438631
distance from User_32 is 1.0437218
distance from User_45 is 0.93731964
distance from User_42 is 0.99616885
distance from User_60 is 0.8746288
distance from User_39 is 1.1123046
distance from User_40 is 0.9685566
distance from User_49 is 0.8049005
distance from User_26 is 0.9123302
distance from User_14 is 1.0393697
distance from User_31 is 1.0420641
distance from User_57 is 1.077325
distance from User_56 is 1.28986
distance from User_11 is 1.1993849
distance from User_51 is 1.199887
distance from User_23 is 0.6572484
distance from User_3 is 1.2439194
distance from User_55 is 1.2333232
distance from User_59 is 1.0046676
distance from User_54 is 1.1205921
distance from User_58 is 1.1783271
distance from User_7 is 1